In [1]:
import numpy as np
import pandas as pd

In [2]:
#importing necessary libraries
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


In [14]:
#converting csv file to pandas dataframe
df = pd.read_csv('1.csv')
#selecting only relevant columns and dropping the duplicate values
df = df[['name','rate','location','dish_liked','cuisines','approx_cost','neighbourhood']]
df.drop_duplicates('name', inplace=True)
df.dropna(subset=['cuisines'], inplace=True)

ERROR! Session/line number was not unique in database. History logging moved to new session 87


In [15]:
#removing punctuations and creating a new column 'addr' by combining 'location' and 'neighbourhood' 
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["cuisines"] = df["cuisines"].apply(lambda text: remove_punctuation(text))
df['addr'] = df['neighbourhood'] +' '+ df['location']

In [16]:
df

,name,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
0,Jalsa,4.1,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",North Indian Mughlai Chinese,800.0,Banashankari,Banashankari Banashankari
1,Spice Elephant,4.1,Banashankari,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",Chinese North Indian Thai,800.0,Banashankari,Banashankari Banashankari
2,San Churro Cafe,3.8,Banashankari,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",Cafe Mexican Italian,800.0,Banashankari,Banashankari Banashankari
3,Addhuri Udupi Bhojana,3.7,Banashankari,Masala Dosa,South Indian North Indian,300.0,Banashankari,Banashankari Banashankari
4,Grand Village,3.8,Basavanagudi,"Panipuri, Gol Gappe",North Indian Rajasthani,600.0,Banashankari,Banashankari Basavanagudi
...,...,...,...,...,...,...,...,...
51544,Chime - Sheraton Grand Bengaluru Whitefield Ho...,4.3,"ITPL Main Road, Whitefield","Cocktails, Pizza, Buttermilk",Finger Food,2500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
51547,The Nest - The Den Bengaluru,3.4,"ITPL Main Road, Whitefield",NaN,Finger Food North Indian Continental,1500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
51551,Nawabs Empire,3.2,"ITPL Main Road, Whitefield",NaN,North Indian Chinese Arabian Momos,300.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
51569,SeeYa Restaurant,3.3,KR Puram,NaN,North Indian Kerala Chinese,350.0,Whitefield,Whitefield KR Puram


In [17]:
#partitioning the dataset into train and test dataset
train_df = df.iloc[:5000,:]
test_df = df.iloc[5000:,:]
df.set_index('name', inplace=True)
train_df.set_index('name', inplace=True)
test_df.set_index('name', inplace=True)

#a pandas dataframe is created using names as indexes
indices = pd.Series(df.index)

In [8]:
#For converting words to embeddings, the string must be in one-hot encoded form
#3 lists are created, where each list represent 1 column
l = [i for i in df['neighbourhood']]
l2 = [i for i in df['cuisines']]
l3 = [i for i in df['dish_liked']]

dish=[]
for i in l3:
    if pd.isna(i):
        dish.append('')
    else:
        dish.append(''.join(i.split(',')))

# integer encode the documents
vocab_size = 200
encoded_docs = [one_hot(d, vocab_size) for d in l]
encoded_docs2 = [one_hot(d, vocab_size) for d in l2]
encoded_docs3 = [one_hot(d, vocab_size) for d in dish]

#one-hot encoded lists are padded with zeros so that every lists has the same length
max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs2 = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
padded_docs3 = pad_sequences(encoded_docs3, maxlen=max_length, padding='post')

ERROR! Session/line number was not unique in database. History logging moved to new session 85


In [9]:
# neural-network model is defined
model = Sequential()
model.add(Embedding(vocab_size, 25, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 25)            5000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 251       
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# fit the model
embd = model.predict(padded_docs)
embd2 = model.predict(padded_docs2)
embd3 = model.predict(padded_docs3)

In [11]:
embd,embd2,embd3

(array([[0.49930283],
        [0.49930283],
        [0.49930283],
        ...,
        [0.49947673],
        [0.49947673],
        [0.49947673]], dtype=float32),
 array([[0.49367636],
        [0.5019616 ],
        [0.4952883 ],
        ...,
        [0.48930046],
        [0.48692673],
        [0.4945088 ]], dtype=float32),
 array([[0.49692136],
        [0.5007732 ],
        [0.50788665],
        ...,
        [0.49898437],
        [0.49898437],
        [0.49898437]], dtype=float32))

In [18]:
# a new matrix is created where each row represents 1 restaurant and the columns represent the Location, 
#Cuisine, Dishes_liked encodings
sim=[]
for i in range(indices.shape[0]):
    sim.append([embd[i][0],embd2[i][0],embd3[i][0]])

In [19]:
sim[0]

[0.49930283, 0.49367636, 0.49692136]

In [58]:
#helper function to find the Pearson's Correlation coefficient between 2 numpy arrays
def correlation_coefficient(T1, T2):
    numerator = np.mean((T1 - T1.mean()) * (T2 - T2.mean()))
    denominator = T1.std() * T2.std()
    if denominator == 0:
        return 0
    else:
        result = numerator / denominator
        return result

# Main function that takes name of a restaurant as input and returns lists of similar restaurants using 
# Cosine similarity and Pearson's Coefficient
def rec(name):
    idx = indices[indices == name].index[0]
    corr,pear = [],[]
    train_sim = sim[:5000]
    for i in range(len(train_sim)):
        #corr.append( (cosine_similarity([sim[idx]],[sim[i]])[0][0], i))
        cs = cosine_similarity([sim[idx]],[train_sim[i]])[0][0]
        p =  correlation_coefficient(np.array(sim[idx]),np.array(train_sim[i]))
        if cs>=1:
            corr.append((cs,i))
        if p>=0.99995:
            pear.append((p,i))
     
    corr = sorted(corr,key = lambda x:x[0], reverse=True)
    pear = sorted(pear,key = lambda x:x[0], reverse=True)
    corr = [c[1] for c in corr]
    pear = [c[1] for c in pear]
    if idx in corr:corr.remove(idx)
    if idx in pear:pear.remove(idx)
        
    return corr,pear

In [60]:
# For every restaurant in test dataset(1000), recommnedations are generated from the train dataset(5000) 
# Evaluation metrics - Quality and Coverage for this model is calculated
psum =0
csum = 0
cset,pset = set(),set()
for ind in test_df.index[:1000]:
    cosine,pearson = rec(ind)
    csum += len(cosine)
    psum += len(pearson)
    for i in cosine:
        cset.add(i)
    for i in pearson:
        pset.add(i)

print("Cosine Quality")
print(csum/1000)
print("Pearson Quality")
print(psum/1000)
print()
print("Cosine Coverage")
print(len(cset)/len(train_df))
print("Pearson Coverage")
print(len(pset)/len(train_df))

Cosine Quality
8.278
Pearson Quality
30.396

Cosine Coverage
0.2922
Pearson Coverage
0.774
